In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# 1. Load and Preprocess Data
# ---------------------------------------------------
# Load CSV; assume first column is label and the rest are features.
data = pd.read_csv('../svm model/edge_hog_features.csv', header=None)
labels = data.iloc[:, 0].values  # First column: labels
features = data.iloc[:, 1:].values  # Remaining columns: features

# Convert labels to strings (or int if needed)
labels = labels.astype(str)

# Filter out classes with fewer than 2 samples.
label_counts = pd.Series(labels).value_counts()
valid_labels = label_counts[label_counts >= 2].index
filtered_indices = [i for i, label in enumerate(labels) if label in valid_labels]
features = features[filtered_indices]
labels = labels[filtered_indices]

# Impute missing values using mean
imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)

# Normalize features using StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Convert labels to categorical (first factorize to integers)
labels_int, classes = pd.factorize(labels)
num_classes = len(classes)
labels_cat = to_categorical(labels_int, num_classes=num_classes)

# 2. Split Data into Training and Testing Sets
# ---------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    features, labels_cat, test_size=0.2, random_state=42, stratify=labels)

# 3. Build the ANN Model
# ---------------------------------------------------
# Here we define a simple ANN with two hidden layers.
model = Sequential([
    Dense(256, activation='relu', input_shape=(features.shape[1],), kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# 4. Train the Model
# ---------------------------------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint("best_ann_model_4.h5", monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
]

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    batch_size=64,
                    callbacks=callbacks,
                    verbose=1)

# Save final model (if needed)
model.save("best_ann_model_4.h5")
print("Model saved as best_ann_model_4.h5")

# 5. Evaluate the Model on the Test Set
# ---------------------------------------------------

# Evaluate the model on the training set
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy:", train_acc)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_acc)

y_pred = model.predict(X_test).argmax(axis=1)
y_true = y_test.argmax(axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


2025-04-10 12:27:01.678347: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 12:27:01.716439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744268221.763007   52392 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744268221.776500   52392 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744268221.808647   52392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     2,073,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,110,223 (8.05 MB)

 Trainable params: 2,109,455 (8.05 MB)

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9895 - loss: 0.1082 - val_accuracy: 0.7917 - val_loss: 0.7324 - learning_rate: 0.0010
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9910 - loss: 0.0731 - val_accuracy: 0.7917 - val_loss: 0.7152 - learning_rate: 0.0010
Epoch 9/100
13/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9956 - loss: 0.0622

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9953 - loss: 0.0630 - val_accuracy: 0.8042 - val_loss: 0.6923 - learning_rate: 0.0010
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9975 - loss: 0.0500 - val_accuracy: 0.7958 - val_loss: 0.6756 - learning_rate: 0.0010
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9978 - loss: 0.0399 - val_accuracy: 0.8000 - val_loss: 0.6732 - learning_rate: 0.0010
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9980 - loss: 0.0414 - val_accuracy: 0.7875 - val_loss: 0.6769 - learning_rate: 0.0010
Epoch 13/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9979 - loss: 0.0340 - val_accuracy: 0.7875 - val_loss: 0.6820 - learning_rate: 0.0010
Epoch 14/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9967 - loss: 0.0343 - val_accuracy: 0.7917 - val_loss: 0.6975 - learning_rate: 0.0010
Epoch 15/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9998 - loss: 0.0322


Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9997 - loss: 0.0319 - val_accuracy: 0.8083 - val_loss: 0.6929 - learning_rate: 0.0010
Epoch 16/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9977 - loss: 0.0276 - val_accuracy: 0.8083 - val_loss: 0.6802 - learning_rate: 5.0000e-04
Epoch 17/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9996 - loss: 0.0243 - val_accuracy: 0.7958 - val_loss: 0.6736 - learning_rate: 5.0000e-04
Epoch 18/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 0.0182 - val_accuracy: 0.8000 - val_loss: 0.6723 - learning_rate: 5.0000e-04
Epoch 19/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 1.0000 - loss: 0.0211 - val_accuracy: 0.8000 - val_loss: 0.6697 - learning_rate: 5.0000e-04
Epoch 20/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 1.0000 - loss: 0.0126 - val_accuracy: 0.8000 - val_loss: 0.6702 - learning_rate: 5.0000e-

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 1.0000 - loss: 0.0111 - val_accuracy: 0.8125 - val_loss: 0.6732 - learning_rate: 2.5000e-04
Epoch 26/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9985 - loss: 0.0160 - val_accuracy: 0.8083 - val_loss: 0.6760 - learning_rate: 2.5000e-04
Epoch 27/100
14/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 0.0137
Epoch 27: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0137 - val_accuracy: 0.8083 - val_loss: 0.6771 - learning_rate: 2.5000e-04


Model saved as best_ann_model_4.h5
Training Accuracy: 1.0
Test Accuracy: 0.800000011920929
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.62      0.62        16
           1       0.60      0.56      0.58        16
           2       0.93      0.88      0.90        16
           3       0.82      0.88      0.85        16
           4       0.77      0.62      0.69        16
           5       0.80      1.00      0.89        16
           6       0.71      0.62      0.67        16
           7       0.90      0.56      0.69        16
           8       0.68      0.94      0.79        16
           9       0.71      0.62      0.67        16
          10       0.88      0.94      0.91        16
          11       0.82      0.88      0.85        16
          12       1.00      1.00      1.00        16
          13       0.79      0.94      0.86        16
          14       1.00      0.94   